In [32]:
import numpy as np
import math
import numpy as np
import random

LEFT = 0
DOWN = 1
RIGHT = 2
UP = 3

N = 5

# генерирует поле (есть путь от начала до цели)
def generate_random_map(size = N, p = 0.8):
    # size: размер квадратного поля
    # p: вероятность того, что клетка заморожена
        
    valid = False

    # проверяет, есть ли хотя бы один путь до цели
    def is_valid(res):
        frontier = []
        discovered = set()
        frontier.append((0,0)) # добавляем начало пути
        while frontier:
            r, c = frontier.pop()
            if not (r,c) in discovered:
                discovered.add((r,c))
                directions = [(1, 0), (0, 1), 
                (-1, 0), (0, -1)]
                for x, y in directions:
                    r_new = r + x
                    c_new = c + y
                    if r_new < 0 or r_new >= size or c_new < 0 or c_new >= size:
                    # если вышли 
                    # за границы поля, то игнорируем ход
                        continue
                    if res[r_new][c_new] == 1: 
                    # если смогли 
                    # достичь цели - проверка окончена
                        return True
                    if (res[r_new][c_new] != -1):  
                    # если 
                    # не попали в лунку - продолжаем 
                    # ходить по полю
                        frontier.append((r_new, c_new))
        return False # если так и не нашли цели

    while not valid: # генерирует карты,
    # пока не получим подходящую
        p = min(1, p)
        res = np.random.choice([0., -1.], (size, size), p = [p, 1-p])
        res[0][0] = 0.
        res[-1][-1] = 1.
        valid = is_valid(res)
    return res

def generate_map_and_P(size = N):
# сопоставляет каждому состоянию множество четверок: 
# (вероятность перехода, новое состояние,
# награда, ?терминальное состояние)
    desc = generate_random_map(size = N)

    P = {s : {a : [] for a in range(4)} for s in range(size * size)}

    def to_s(row, col):  # преобразует координаты 
    # поля в номер клетки
        return row * size + col

    def inc(row, col, a):  # делаем ход 
    # в зависимости от управления
        if a == LEFT:
            col = max(col - 1, 0)
        elif a == DOWN:
            row = min(row + 1, size - 1)
        elif a == RIGHT:
            col = min(col + 1, size - 1)
        elif a == UP:
            row = max(row - 1, 0)
        return (row, col)

    for row in range(size):
        for col in range(size):
            s = to_s(row, col)
            for a in range(4):
                li = P[s][a]
                letter = desc[row, col]
                if letter in [-1, 1]:
                    li.append((1.0, s, 0, True))
                else:
                    for b in [(a - 1) % 4, a, (a + 1) % 4]:
                        newrow, newcol = inc(row, col, b)
                        newstate = to_s(newrow, newcol)
                        newletter = desc[newrow, newcol]
                        done = newletter == 1
                        if newletter == 1:
                            rew = 1.
                        elif newletter == -1:
                            newstate = s
                        else:
                            rew = 0.
                        if (newstate == s) and  (newstate != 1):
                        # остались на месте, значит ударились, получаем штраф
                            rew = -0.75 
                        if b == a:
                            li.append((0.75, newstate, rew, done))
                        else:
                            li.append((0.125, newstate, rew, done))
                        
    return P, desc

# симулирует шаг из состояния state при выборе действия act
def do_step(state, act):
    e = np.random.uniform(0, 1)
    if e < 0.75:
        return P[state][act][1]
    elif e < 0.875:
        return P[state][act][0]
    else:
        return P[state][act][2]

########################################################

# общие параметры и генерация сетки
gamma = 0.8 # скидка
eps = 1e-10 # порог обучения

P, U = generate_map_and_P()
U1 = np.copy(U)
print(U)
print()

[[ 0. -1. -1.  0.  0.]
 [ 0.  0.  0.  0.  0.]
 [-1.  0.  0.  0. -1.]
 [-1. -1. -1.  0.  0.]
 [ 0.  0.  0.  0.  1.]]



In [33]:
U = U.reshape(N * N)
terms = [] # терминальные состояния
for i in range(N * N):
    if U[i] in [-1, 1]:
        terms.append(i)
        if U[i] in [-1, 1]:
            U[i] = 0   # в таблице со значениями 
            # функции Беллмана в терминальных состояниях 
            # пишем 0, чтобы не суммировать штрафы и 
            # награду 2 раза

# Q-обучение

alpha = 0.01 # скорость обучения

Q_matrix = np.zeros(N * N * 4).reshape(25, 4).astype(np.float32)
print(U1)
print()

all_states = []
for state in range(N * N):
    if state not in terms:
        all_states.append(state)

# процесс заполнения Q-таблицы
for episode in range(100000):
    #s = random.choice(all_states)
    s = 0
    for i in range(20):
        a = random.choice([0, 1, 2, 3])
        _, s_new, rew, done = do_step(s, a)
        Q_update = rew + gamma * np.max(Q_matrix[s_new, :])
        Q_matrix[s, a] = (1 - alpha) * Q_matrix[s, a] + alpha * Q_update
        if done:
            break
        else:
            s = s_new

print(Q_matrix)

U1 = U1.reshape(N * N)
policy = []
for state in range(N * N):
    if U1[state] == -1:
        policy.append('W')
        continue
    elif U1[state] == 1:
        policy.append('G')
        continue
    optimal_act = np.argmax(Q_matrix[state])
    if optimal_act == 0:
        policy.append('L')
    elif optimal_act == 1:
        policy.append('D')
    elif optimal_act == 2:
        policy.append('R')
    elif optimal_act == 3:
        policy.append('U')

policy = np.reshape(policy, [N, N])
print()
print("Optimal policy:")
print(policy)

[[ 0. -1. -1.  0.  0.]
 [ 0.  0.  0.  0.  0.]
 [-1.  0.  0.  0. -1.]
 [-1. -1. -1.  0.  0.]
 [ 0.  0.  0.  0.  1.]]

[[-0.91864395 -0.36109677 -0.953498   -1.0406414 ]
 [ 0.          0.          0.          0.        ]
 [ 0.          0.          0.          0.        ]
 [-0.6384234  -0.01410756 -0.11151494 -0.693632  ]
 [-0.12384152 -0.10780302 -0.7405208  -0.7497409 ]
 [-0.79003733 -0.77169245 -0.18713766 -0.33669662]
 [-0.18821898 -0.03282108 -0.06438587 -0.5605558 ]
 [-0.10292292  0.06682314  0.03573024 -0.50215095]
 [ 0.06129345  0.16039337  0.02328513 -0.00230656]
 [-0.01454089 -0.66146076 -0.6999171  -0.17958279]
 [ 0.          0.          0.          0.        ]
 [-0.6771416  -0.64450794 -0.04439958 -0.10129524]
 [-0.11141884 -0.46252334  0.0786118   0.05452007]
 [ 0.1146354   0.25396046 -0.39681655  0.04085151]
 [ 0.          0.          0.          0.        ]
 [ 0.          0.          0.          0.        ]
 [ 0.          0.          0.          0.        ]
 [ 0.          0